In [129]:
import pandas as pd
import re

In [4]:
#Задание 1

'''Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений'''
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [12]:
def classifaer(row):
    if (row['traffic_source'] =='yandex')|(row['traffic_source'] =='google'):
        return "organic"
    elif (row['region']=='Russia')&((row['traffic_source']=='paid')|(row['traffic_source']=='email')):
        return "ad"
    elif (row['region']!='Russia')&((row['traffic_source']=='paid')|(row['traffic_source']=='email')):
        return "other"
    else:
        return row['traffic_source']
    
    
log['sourse_type'] = log.apply(classifaer, axis=1)
log.sourse_type.value_counts()

direct     7013
organic    5407
ad         3790
other      2728
Name: sourse_type, dtype: int64

In [188]:
#Задание 2

'''В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами
новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. 
Выполните следующие действия:

Прочитайте содержимое файла с датафрейм. Отфильтруйте страницы с текстом новостей, используя метод str.contains 
и регулярное выражение в соответствии с заданным шаблоном'''
news = pd.read_csv('URLs.txt', names=['new',])

news.head()


,new
0,url
1,/world/
2,/latest/
3,/?updated=top
4,/politics/36188461-s-marta-zhizn-rossiyan-susc...


In [189]:
pattern=r'^\/\w+\/\d{8}-.*\/$'

In [190]:
news[news.new.str.contains(pattern)].head()

,new
4,/politics/36188461-s-marta-zhizn-rossiyan-susc...
5,/world/36007585-tramp-pridumal-kak-reshit-ukra...
6,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
7,/video/36001498-poyavilis-pervye-podrobnosti-g...
11,/middleeast/36131117-divizion-s-400-ne-zametil...


In [82]:
# Задание 3

'''Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.'''
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [167]:
#1ый способ решения
ratings_grouped = ratings.groupby('userId').agg({'timestamp':[min,max,'count']})
ratings_grouped.head()
ratings_grouped['diff'] = ratings_grouped['timestamp']['max']- ratings_grouped['timestamp']['min'] 
ratings_grouped[ratings_grouped['timestamp']['count']>100]['diff'].mean()

40080507.4496124

In [154]:
#2ой способ решения, который не получилось реализовать. Идея была - сначала получить новый df, в котором оставлены только те 
#userId, которые выставляли более 100 оценок.
#ratings.groupby('userId').count()>100 - условие фильтрации
#не понимаю как сделать фильтрацию по этому условию и получить новый df

In [71]:
# функция для работы с новым df 
def groupby_function(data):
    return data.timestamp.max() - data.timestamp.min()
(ratings.groupby('userId').apply(groupby_function)).mean()

17373661.00447094

In [52]:
#Задание 4

'''Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.'''
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [54]:
joined = rzd.merge(auto, on='client_id', how='outer')
joined1 = joined.merge(air, on='client_id', how='outer' )
joined1.head()

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0


In [56]:
joined2 = joined1.merge(client_base, on='client_id', how='outer' )
joined2.head()

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
